## Логическая модель данных:

<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/SwimmersDB.png?raw=1" />

<b>_Представление_</b> cоздает виртуальную таблицу, содержимое которой определяется запросом.<br/>
Используется":

- Для упрощения и особой формы представления информации в БД для каждого пользователя;
- В качестве механизма безопасности, позволяющего пользователям обращаться к данным через представления;
- Для предоставления интерфейса обратной совместимости, моделирующего таблицу, схема которой изменилась.

Синтаксис:

```
-- Syntax for SQL Server and Azure SQL Database  
  
CREATE [ OR ALTER ] VIEW [ schema_name . ] view_name [ (column [ ,...n ] ) ]   
[ WITH <view_attribute> [ ,...n ] ]   
AS select_statement   
[ WITH CHECK OPTION ]   
[ ; ]  
  
<view_attribute> ::=   
{  
    [ ENCRYPTION ]  
    [ SCHEMABINDING ]  
    [ VIEW_METADATA ]       
}   
```

Ограничения:

    - An ORDER BY clause, unless there is also a TOP clause in the select list of the SELECT statement

        Important

        The ORDER BY clause is used only to determine the rows that are returned by the TOP or OFFSET clause in the view definition. The ORDER BY clause does not guarantee ordered results when the view is queried, unless ORDER BY is also specified in the query itself.

    - The INTO keyword

    - The OPTION clause

    - A reference to a temporary table or a table variable.



You can modify the data of an underlying base table through a view, as long as the following conditions are true:

- Any modifications, including UPDATE, INSERT, and DELETE statements, must reference columns from only one base table.

- The columns being modified in the view must directly reference the underlying data in the table columns. The columns cannot be derived in any other way, such as through the following:

    An aggregate function: AVG, COUNT, SUM, MIN, MAX, GROUPING, STDEV, STDEVP, VAR, and VARP.

    A computation. The column cannot be computed from an expression that uses other columns. Columns that are formed by using the set operators UNION, UNION ALL, CROSSJOIN, EXCEPT, and INTERSECT amount to a computation and are also not updatable.

- The columns being modified are not affected by GROUP BY, HAVING, or DISTINCT clauses.

- TOP is not used anywhere in the select_statement of the view together with the WITH CHECK OPTION clause.



If the previous restrictions prevent you from modifying data directly through a view, consider the following options:

- INSTEAD OF Triggers

    INSTEAD OF triggers can be created on a view to make a view updatable. The INSTEAD OF trigger is executed instead of the data modification statement on which the trigger is defined. This trigger lets the user specify the set of actions that must happen to process the data modification statement. Therefore, if an INSTEAD OF trigger exists for a view on a specific data modification statement (INSERT, UPDATE, or DELETE), the corresponding view is updatable through that statement. For more information about INSTEAD OF triggers, see DML Triggers.

- Partitioned Views

    If the view is a partitioned view, the view is updatable, subject to certain restrictions. When it is needed, the Database Engine distinguishes local partitioned views as the views in which all participating tables and the view are on the same instance of SQL Server, and distributed partitioned views as the views in which at least one of the tables in the view resides on a different or remote server.


Пример #1:

In [0]:
use tempdb
go

if OBJECT_ID('dbo.v_Swimmers') is not null
   drop view dbo.v_Swimmers
go

-----------------------------------------------------------------------------------
-- view returns all swimmers 
-- created by:   Timofey Gavrilenko
-- created date: 4/29/2019
-- sample call:  
-- select * from dbo.v_Swimmers
-----------------------------------------------------------------------------------

create view dbo.v_Swimmers
as  
    select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
            sc.[Name] Club, sc.City, c.[Name] Category
    from dbo.Swimmer s
    left join dbo.SwimmingClub sc on s.SwimmingClubID = sc.SwimmingClubID
    left join dbo.Category c      on s.CategoryID     = c.CategoryID
go

Обновление данных через представление:

In [7]:
update v_Swimmers
	set --City = N'Минск',
		YearOfBirth = 2006
where FirstName = N'Анна' and LastName = N'Высоцкая'

Код ниже вызовет ошибку 
```
View or function 'v_Swimmers' is not updatable because the modification affects multiple base tables.
```


In [0]:
delete from v_Swimmers
where FirstName = N'Анна' and LastName = N'Высоцкая'

<b>_Хранимые процедуры_</b> похожи на процедуры из других языков программирования в том, что они могут:

- принимать входные параметры и возвращать вызывающей процедуре или пакету ряд значений в виде выходных параметров;

- содержать программные инструкции, которые выполняют операции в базе данных, в том числе вызывающие другие процедуры;

- возвращать значение состояния вызывающей процедуре или пакету, таким образом передавая сведения об успешном или неуспешном завершении (и причины последнего).

Синтаксис:
```
-- Transact-SQL Syntax for Stored Procedures in SQL Server and Azure SQL Database  
  
CREATE [ OR ALTER ] { PROC | PROCEDURE } 
    [schema_name.] procedure_name [ ; number ]   
    [ { @parameter [ type_schema_name. ] data_type }  
        [ VARYING ] [ = default ] [ OUT | OUTPUT | [READONLY]  
    ] [ ,...n ]   
[ WITH <procedure_option> [ ,...n ] ]  
[ FOR REPLICATION ]   
AS { [ BEGIN ] sql_statement [;] [ ...n ] [ END ] }  
[;]  
  
<procedure_option> ::=   
    [ ENCRYPTION ]  
    [ RECOMPILE ]  
    [ EXECUTE AS Clause ]
```
Подробнее о хранимых процедурах T-SQL:<br/>
https://docs.microsoft.com/ru-ru/sql/t-sql/statements/create-procedure-transact-sql?view=sql-server-2017

Пример #2:

Хранимая процедура, с json-параметром по умолчанию:

In [3]:
use tempdb
go

if OBJECT_ID('dbo.usp_GetSwimmersList', 'P') is not null
   drop procedure dbo.usp_GetSwimmersList
go

------------------------------------------------------------------------------------
-- procedure returns list of Swimmers from given clubs
-- created by:   Timofey Gavrilenko
-- created date: 4/22/2019
-- sample call:  
-- exec dbo.usp_GetSwimmersList @parameters = N'[{"Club": "Трактор", "City": "Минск"}]'
-----------------------------------------------------------------------------------

create procedure dbo.usp_GetSwimmersList
    @parameters nvarchar(1000) = null
as    
begin
    select  s.FirstName,
            s.LastName,
            s.YearOfBirth,
            ISNULL(sc.[Name] + ' ' + sc.City, '-') Club,
            ISNULL(c.[Name], '-') Category
    from dbo.Swimmer s
    left join dbo.SwimmingClub sc   on s.SwimmingClubID = sc.SwimmingClubID
    left join dbo.Category c        on s.CategoryID = c.CategoryID
    outer apply (
        select  [Name], 
				City
        from openjson(@parameters) 
            with (
                     [Name] nvarchar(100) '$.Club',
                     City nvarchar(100) '$.City'
                 ) j
        where (j.[Name] = sc.[Name] or j.[Name] is null) and
              (j.City = sc.City or j.City is null)
    ) f
    where (f.City is not null or f.[Name] is not null) or @parameters is null
end
go

<b>_Скалярная функция_</b> - функция, возвращающая скалярное значение. Как правило, это значение одного из стандартных T-SQL типов (int, varchar(x), datetime и т.д.)

Однако, возвращаемым значением может быть XML, JSON или значение, определенного пользователем типа.<br/>
Значения по умолчанию возможны, но опустить фактический параметр при вызове функции нельзя.

Синтаксис:
```
CREATE  FUNCTION [ owner_name. ] function_name 
    ( [ { @parameter_name [AS] scalar_parameter_data_type [ = default ] } 
      [ ,...n ] ] ) 

RETURNS scalar_return_data_type

[ WITH function_option [ [,] ...n] ] 

[ AS ]

BEGIN 
    function_body 
    RETURN scalar_expression
END
```
Подробнее о функциях T-SQL:
https://docs.microsoft.com/ru-ru/sql/t-sql/statements/create-function-transact-sql?view=sql-server-2017

Пример #3:

In [1]:
use tempdb
go

if OBJECT_ID('dbo.fn_GetSwimmersCount', 'F') is not null
   drop function dbo.fn_GetSwimmersCount
go

------------------------------------------------------------------------------------
-- function returns number of Swimmers for given club (@ClubID)
-- created by:   Timofey Gavrilenko
-- created date: 4/26/2019
-- sample call:  
-- select dbo.fn_GetSwimmersCount(default)
-----------------------------------------------------------------------------------

create function dbo.fn_GetSwimmersCount(
    @ClubID int = null
) returns int
as  
begin
  return (select count(1) from dbo.Swimmer where SwimmingClubId = @ClubID or @ClubID is null)
end
go

Commands completed successfully.

Total execution time: 00:00:00.0015604

Commands completed successfully.

Total execution time: 00:00:00.0012716

Commands completed successfully.

Total execution time: 00:00:00.0164098

<b>_Табличная функция_</b> возвращает набор данных (виртуальную таблицу)


Может быть:
- inline (лучше оптимизируются)
- multi-statement (больше возможностей)

Синтаксис:
```
-- Transact-SQL Inline Table-Valued Function Syntax   
CREATE [ OR ALTER ] FUNCTION [ schema_name. ] function_name   
( [ { @parameter_name [ AS ] [ type_schema_name. ] parameter_data_type   
    [ = default ] [ READONLY ] }   
    [ ,...n ]  
  ]  
)  
RETURNS TABLE  
    [ WITH <function_option> [ ,...n ] ]  
    [ AS ]  
    RETURN [ ( ] select_stmt [ ) ]  
[ ; ]
```
-----------------------------------------------------------------------

```
-- Transact-SQL Multi-Statement Table-Valued Function Syntax  
CREATE [ OR ALTER ] FUNCTION [ schema_name. ] function_name   
( [ { @parameter_name [ AS ] [ type_schema_name. ] parameter_data_type   
    [ = default ] [READONLY] }   
    [ ,...n ]  
  ]  
)  
RETURNS @return_variable TABLE <table_type_definition>  
    [ WITH <function_option> [ ,...n ] ]  
    [ AS ]  
    BEGIN   
        function_body   
        RETURN  
    END  
[ ; ]
```



Пример #4:

In [6]:
use tempdb
go

if OBJECT_ID('dbo.fn_GetSwimmersInline') is not null
   drop function dbo.fn_GetSwimmersInline
go

------------------------------------------------------------------------------------
-- function returns all swimmers for given club (@ClubID)
-- created by:   Timofey Gavrilenko
-- created date: 4/26/2019
-- sample call:  
-- select * from dbo.fn_GetSwimmersInline(N'ж', 2006)
-----------------------------------------------------------------------------------

create function dbo.fn_GetSwimmersInline(
    @Gender nchar(1) = null,
    @YearOfBirth int = null
) returns table
as  
  return (
      select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
             sc.[Name] Club, sc.City, c.[Name] Category
      from dbo.Swimmer s
      left join dbo.SwimmingClub sc on s.SwimmingClubID = sc.SwimmingClubID
      left join dbo.Category c      on s.CategoryID     = c.CategoryID
      where (Gender = @Gender            or @Gender is null) and
            (YearOfBirth = @YearOfBirth  or @YearOfBirth is null)
 )
go

Пример #5:

In [0]:
use tempdb
go

if OBJECT_ID('dbo.fn_GetSwimmersMultyStatement') is not null
   drop function dbo.fn_GetSwimmersMultyStatement
go

------------------------------------------------------------------------------------
-- function returns all swimmers for given club (@ClubID)
-- created by:   Timofey Gavrilenko
-- created date: 4/26/2019
-- sample call:  
-- select * from dbo.fn_GetSwimmersMultyStatement(N'ж', 2006)
-----------------------------------------------------------------------------------

create function dbo.fn_GetSwimmersMultyStatement(
    @Gender nchar(1) = null,
    @YearOfBirth int = null
) returns @Swimmers table
(
    SwimmerID       int             not null,
    FirstName       nvarchar(20)    not null,
    LastName        nvarchar(30)    not null,
    YearOfBirth     smallint        not null,
    Gender          nchar(1)        not null,
    Club            nvarchar(100), 
	City            nvarchar(30), 
    Category        nvarchar(20)    
)
as  
begin
  insert into @Swimmers (SwimmerID, FirstName, LastName, YearOfBirth, Gender, Club, City, Category)
      select SwimmerID, FirstName, LastName, YearOfBirth, Gender, sc.[Name] Club, sc.City, c.[Name] Category
      from dbo.Swimmer s
      left join dbo.SwimmingClub sc on s.SwimmingClubID = sc.SwimmingClubID
      left join dbo.Category c      on s.CategoryID     = c.CategoryID
      where (Gender = @Gender            or @Gender is null) and
            (YearOfBirth = @YearOfBirth  or @YearOfBirth is null)

  return
end
go

<b>DML-тригггер AFTER (FOR) и INSTEAD OF</b> - особая разновидность хранимой процедуры, которая автоматически выполняется при возникновении события на сервере базы данных. Триггеры DML выполняются, когда пользователь пытается изменить данные с помощью событий языка обработки данных (DML). Событиями DML являются выполнение операций INSERT, UPDATE или DELETE в применении к таблице или представлению. 

Синтаксис:
```
-- SQL Server Syntax  
-- Trigger on an INSERT, UPDATE, or DELETE statement to a table or view (DML Trigger)  
  
CREATE [ OR ALTER ] TRIGGER [ schema_name . ]trigger_name   
ON { table | view }   
[ WITH <dml_trigger_option> [ ,...n ] ]  
{ FOR | AFTER | INSTEAD OF }   
{ [ INSERT ] [ , ] [ UPDATE ] [ , ] [ DELETE ] }   
[ WITH APPEND ]  
[ NOT FOR REPLICATION ]   
AS { sql_statement  [ ; ] [ ,...n ] | EXTERNAL NAME <method specifier [ ; ] > }  
  
<dml_trigger_option> ::=  
    [ ENCRYPTION ]  
    [ EXECUTE AS Clause ]  
  
<method_specifier> ::=  
    assembly_name.class_name.method_name
```

Пример #6:

Необходимо предварительно выполнить настройку email-профиля и следующий код:
```
sp_configure 'show advanced options', 1;  
go  
reconfigure;  
go  
sp_configure 'Database Mail XPs', 1;  
go  
reconfigure  
go  
```

In [0]:
use tempdb
go

if OBJECT_ID('dbo.t_Swimmer_Notification') is not null
   drop view dbo.t_Swimmer_Notification
go

-----------------------------------------------------------------------------------
-- trigger sends email-notifications whenever dbo.Swimmer table gets updated
-- created by:   Timofey Gavrilenko
-- created date: 4/29/2019
-----------------------------------------------------------------------------------
create trigger t_Swimmer_Notification
on dbo.Swimmer
after insert, update, delete
as  
begin
   declare @_records_affected varchar(max)
   if exists(select 1 from deleted)
        select top 3 @_records_affected += LastName + ' ' + FirstName + ', '
        from deleted
   else
        select top 3 @_records_affected += LastName + ' ' + FirstName
        from inserted
   set @_records_affected += '...'

   declare @_body varchar(max)
   set @_body = formatmessage('There are changes to dbo.Swimmer table [%s]. Made by: [%s]', @_records_affected, user_name())

   exec msdb.dbo.sp_send_dbmail  
        @profile_name = 'system',  
        @recipients = 'manager@swimmer.com',  
        @body = @_body,  
        @subject = 'notification';  
end        
go

Пример #7:

INSTEAD OF - триггер для модифицикации многотабличных представлений или альтернативного исполнения инструкций DML

In [0]:
use tempdb
go
--patch for implementing soft deletes

--01. modifications to dbo.Swimmer table
if not exists(
    select 1 
    from INFORMATION_SCHEMA.COLUMNS 
    where TABLE_SCHEMA = 'dbo' and TABLE_NAME = 'Swimmer' and COLUMN_NAME = 'IsDeleted'
    )
begin    
	alter table dbo.Swimmer add IsDeleted bit not null constraint DF_Swimmer_IsDeleted default (0) 
end
go
    
--02. new version of v_Swimmer view (to be able to handle soft deletes)
if OBJECT_ID('dbo.v_Swimmers') is not null
   drop view dbo.v_Swimmers
go

create view dbo.v_Swimmers
as  
    select s.SwimmerID, s.FirstName, s.LastName, s.YearOfBirth, s.Gender, 
            sc.[Name] Club, sc.City, c.[Name] Category
    from dbo.Swimmer s
    left join dbo.SwimmingClub sc on s.SwimmingClubID = sc.SwimmingClubID
    left join dbo.Category c      on s.CategoryID     = c.CategoryID
	where IsDeleted = 0
go


--03. trigger to replace hard-delete with its soft version
if OBJECT_ID('dbo.t_Swimmer_Delete') is not null
   drop trigger dbo.t_Swimmer_Delete
go

create trigger t_Swimmer_Delete
on dbo.Swimmer
instead of delete
as
begin
	set nocount on

	update s
	set IsDeleted = 1
	from dbo.Swimmer s
	inner join deleted d on s.SwimmerID = d.SwimmerID
	where s.IsDeleted = 0
end
go


Использование:

In [0]:
select * from dbo.Swimmer

delete from dbo.Swimmer
where Gender = N'ж'

select * from dbo.v_Swimmers

select * from dbo.Swimmer